# Démonstration

Ce notebook permet de prédire la famille d'un article à partir du moment de la journée, du jour de la semaine et du type de consommateur.

# Classification

In [75]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.feature_selection import VarianceThreshold
from sklearn.cluster import KMeans
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier



data_folder = os.path.join(os.getcwd(), '../data_original')
df_correct = pd.read_csv(os.path.join(data_folder, 'correct_data.csv'), sep=',')
df_user = pd.read_csv(os.path.join(data_folder, 'users.csv'), sep=',')

# 1. Clustering

In [76]:
# On normalise toutes les colonnes sauf l'ID acheteur
X = df_user[df_user.columns[1:]].values

scaler = RobustScaler()
X = scaler.fit_transform(X)

df_user[df_user.columns[1:]] = X

In [77]:
column_names = df_user.columns[1:]

selector = VarianceThreshold(threshold=0.2)
X = selector.fit_transform(X)

In [78]:
model = PCA(n_components=15)
X = model.fit_transform(X)

In [79]:
model = KMeans(n_clusters=3, n_init="auto")
labels = model.fit_predict(X)

In [80]:
df_user['label'] = labels

In [81]:
df_correct = df_correct.loc[df_correct.index.repeat(df_correct['Quantité'])]


In [82]:
df_correct.drop(columns=['Date/heure transaction', 'Article', 'Prix unitaire TTC', 'Quantité', 'Total TTC', 'Semestre', 'Automne/Printemps'], inplace=True)

In [83]:
df_correct = df_correct.merge(df_user[['ID acheteur', 'label']], on='ID acheteur', how='left')
df_correct.drop(columns=['ID acheteur'], inplace=True)

In [84]:
# One hot encoding sur la periode, le jour de la semaine, le label avec get_dummies
df_correct = pd.get_dummies(df_correct, columns=['Periode', 'Jour semaine', 'label'])

In [85]:
# labelencoding
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df_correct['Famille d\'article'] = le.fit_transform(df_correct['Famille d\'article'])

df_correct.head()

,Famille d'article,Periode_apresmidi,Periode_matin,Periode_midi,Periode_soir,Jour semaine_jeudi,Jour semaine_lundi,Jour semaine_mardi,Jour semaine_mercredi,Jour semaine_vendredi,label_0,label_1,label_2
0,7,True,False,False,False,False,False,False,False,True,False,False,True
1,7,True,False,False,False,False,False,False,False,True,False,False,True
2,7,True,False,False,False,False,False,False,False,True,False,False,True
3,7,True,False,False,False,False,False,False,False,True,False,False,True
4,7,True,False,False,False,False,False,False,False,True,False,False,True


In [86]:
# modeling
from sklearn.model_selection import train_test_split

X = df_correct[df_correct.columns[1:]]
y = df_correct[df_correct.columns[0]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [87]:
# sgdc
model = SGDClassifier(alpha=0.1, fit_intercept=True, loss='modified_huber', n_jobs=-1, penalty='l2')
model.fit(X_train, y_train)

SGDClassifier(alpha=0.1, loss='modified_huber', n_jobs=-1)

In [88]:
# On créé un exemple de vente (df avec une seule ligne)
df_example = pd.DataFrame(columns=df_correct.columns[1:])

ll = ["Jour semaine", "Periode", "label"]
periode_journee = ["matin", "midi", "apresmidi", "soir"]
jour_semaine = ["lundi", "lardi", "mercredi", "jeudi", "vendredi"]
label = [0, 1, 2]

res_liste = dict()
# On ajoute les valeurs dans le df
for l in ll:
    res = None
    if l == "Periode":
        while res not in periode_journee:
            res = input("Entrez la période (matin, midi, apresmidi, soir) : ")
        
        res_liste[l + "_" + res] = True  
        
        
    elif l == "Jour semaine":
        while res not in jour_semaine:
            res = input("Entrez le jour de la semaine (lundi, mardi, mercredi, jeudi, vendredi) : ")
        
        res_liste[l + "_" + res] = True
        
    else:
        while res not in label:
            res = int(input("Entrez le label (0, 1, 2) : "))
        
        res_liste[l + "_" + str(res)] = True


# On transforme le dictionnaire en dataframe
for k in res_liste.keys():
    df_example[k] = [res_liste[k]]

df_example.fillna(False, inplace=True)

In [89]:
# On effectue la prédiction de la famille d'article
pred = model.predict(df_example)
print("La famille d'article prédite est : ", le.inverse_transform(pred)[0])

La famille d'article prédite est :  Snacks Sucrés


L'exemple précédent correspond à un achat d'un petit consommateur le jeudi après-midi.